## Phasing of single diffraction pattern (Fig. 11)

In [1]:
# Use autoreload
%load_ext autoreload
%autoreload 2

# Import modules
import sys, h5py
import numpy as np
import scipy.ndimage as ndimage
import spimage
%matplotlib inline

# Import modules from src directory
sys.path.append("../src")
import plotting

In [2]:
# Configuration (Experiment)
distance_back  = 2.4 #[m]
distance_front = 0.497 #[m]
pixelsize = 110e-6 #[m]
wavelength = 0.2262e-9 #[m]
downsampling = 4

### 1. Loading data from file

In [ ]:
# Loading the signal (assembled)
filename = "../meta/phasing.h5"
N = 5000
with h5py.File(filename, 'r') as f:
    recons  = f['real_space_final'][:N]
    fourier = f['fourier_space_final'][:N]
    support = f['support_final'][:N]
    rerror  = f["real_error"][:N]
    ferror  = f["fourier_error"][:N]

### 2. Validation using PRTF

In [ ]:
valid = (ferror < 0.2151) & (rerror < 0.1402)
print valid.sum()
print recons.shape

In [ ]:
# Calculate PRTF
output_prtf = spimage.prtf(recons, support, enantio=True, translate=True)

In [ ]:
# Collect PRTF results
superimage = output_prtf['super_image']
prtf_2d    = output_prtf['prtf']

In [ ]:
# Save to file
with h5py.File('../analysis/phasing/prtf.h5', 'w') as f:
    f['super_image'] = superimage
    f['prtf'] = prtf_2d

In [ ]:
nx, ny = prtf_2d.shape[1], prtf_2d.shape[0]
xx,yy = np.meshgrid(np.arange(nx),np.arange(ny))
mask_radial = np.sqrt((xx-nx/2)**2 + (yy-ny/2)**2) < nx/2
prtf_centers, prtf_radial = spimage.radialMeanImage(prtf_2d, msk=mask_radial, output_r=True)
prtf_qr = x_to_q(prtf_centers, distance_front)
limit_qr = prtf_qr[np.abs(ndimage.gaussian_filter1d(prtf_radial,2) - (1/np.e)).argmin()]

In [ ]:
# Pixelsize in real space
dx = wavelength * distance_front / (intensities_cropped.shape[0] * pixelsize*downsampling) #[m/px]
print "One pixel in real space corresponds to %.1f nm" %(dx*1e9)
print "The support should have a size (diameter) of about %.2f px" %(signal_diameter / (dx*1e9))

### 3. Plotting

In [ ]:
plot = plotting.Plot(fontsize=8, rows=1, cols=3, legend=True, legend_frameon=False, exclude=(1,2), save_pdf=True)
plot.add_axes((1,0),2,1, padx=0)
plot.xlabel = ['', r'Spatial frequency [nm$^{-1}$]']
plot.ylabel = ['', 'PRTF']
plot.title_label = 2*['']

plot.plotting_a_map(0, np.abs(superimage)[46:-44,45:-45], cmap='magma')
plot.axes[0].set_xticks([])
plot.axes[0].set_xticklabels([])
plot.axes[0].set_yticks([])
plot.axes[0].set_yticklabels([])
plot.plotting_a_rectangle(0, 25, 34, 20./(1e9*dx), 1, facecolor='k', edgecolor='k')
#plot.axes[0].text(15+20./(1e9*dx), 34, '40 nm', ha='center', va='bottom', fontsize=10, color='k')
plot.axes[0].text(-1, -5, '(a)', va='top', ha='left', fontsize=10)

plot.plotting_traces(1, [prtf_qr[:-5][::4]], 
                     [prtf_radial[:-5].reshape((15,4)).mean(axis=1)], 
                     [''], ylim=[0,1],xlim=[prtf_qr[0],0.2],
                     linestyles=['-'], linewidths=[1], markers=[None],
                     mecs=['k'], mfcs=['k'], colors=['k'])
plot.axes[1].axhline(1/np.e, ls=':', color='k')
plot.axes[1].axvline(1/14., color='k', ls='--')
plot.axes[1].tick_params(labelright='on', labelleft='off')
plot.axes[1].yaxis.set_label_position('right')
plot.axes[1].text(0, 1.1, '(b)', va='top', ha='left', fontsize=10)
plot.axes[1].text(0.18, 1/np.e, '1/e', va='bottom')

plot.save('/Users/benedikt/phd-project/documentation/manuscripts/omrv-paper/manuscript/figures/fig_phasing.pdf')
plot.show()
print "The PRTF drops below 1/e at a resolution of %.2f nm" %(1./limit_qr)

In [ ]:
plot = plotting.Plot(fontsize=8, axes_visible=False, save_png=True)
plot.title_label = ['']
plot.xlabel = ['']
plot.ylabel = ['']
plot.plotting_a_map(0, np.abs(recons[0][32:-32,32:-32]), cmap='magma')
#plot.plotting_a_rectangle(0, 100, 118, 40./(1e9*dx), 2, facecolor='w', edgecolor='w')
#plot.axes[0].text(100+20./(1e9*dx), 116, '40 nm', ha='center', va='bottom', fontsize=12, color='w')
#plot.save('../plots/phasing_recons.png')
plot.show()

In [ ]:
plot = plotting.Plot(fontsize=8, axes_visible=False, save_png=True)
plot.title_label = ['']
plot.xlabel = ['']
plot.ylabel = ['']
plot.plotting_a_map(0, np.abs(superimage)[45:-45,45:-45], cmap='gray_r')
#plot.plotting_a_rectangle(0, 100, 118, 40./(1e9*dx), 2, facecolor='w', edgecolor='w')
#plot.axes[0].text(100+20./(1e9*dx), 116, '40 nm', ha='center', va='bottom', fontsize=12, color='w')
#plot.save('../plots/phasing_recons.png')
plot.show()

In [ ]:
plot = plotting.Plot(fontsize=8, colorbar=True, axes_visible=False, save_png=True)
plot.title_label = ['']
plot.xlabel = ['']
plot.ylabel = ['']
prtf_img = ndimage.gaussian_filter(prtf_2d,2)
plot.plotting_a_map(0, spimage.downsample(prtf_2d,2) , mask=None, log=False, vmin=0, vmax=1, bad='0.8', under='w', cmap='jet')

In [ ]:
# Convert index to q (in nm)
x_to_q = lambda x, d: (2*np.sin(np.arctan2(x*pixelsize*downsampling, d)/2.)) / wavelength / 1e9

In [ ]:
prtf_2d_down = spimage.downsample(prtf_2d,1)
nx, ny = prtf_2d_down.shape[1], prtf_2d_down.shape[0]
xx,yy = np.meshgrid(np.arange(nx),np.arange(ny))
mask_radial = np.sqrt((xx-nx/2)**2 + (yy-ny/2)**2) < nx/2
prtf_centers, prtf_radial = spimage.radialMeanImage(prtf_2d_down, msk=mask_radial, output_r=True)
prtf_qr = x_to_q(prtf_centers, distance_front)
limit_qr = prtf_qr[np.abs(ndimage.gaussian_filter1d(prtf_radial,2) - (1/np.e)).argmin()]
plot = plotting.Plot(fontsize=8, aspect=.5, legend=True, save_pdf=True)
plot.title_label = ['']
plot.xlabel = ['']
plot.ylabel = ['']
plot.plotting_traces(0, [prtf_qr[:-1][::2], prtf_qr], 
                     [prtf_radial[:-1].reshape((32,2)).mean(axis=1), ndimage.gaussian_filter(prtf_radial,2)], 
                     ['data', 'smooth'], ylim=[0,1],
                     linestyles=['-','-'], linewidths=[1,2], markers=[None,None],
                     mecs=['k', 'r'], mfcs=['k', 'r'], colors=['k','r'])
plot.axes[0].axhline(1/np.e)
plot.axes[0].axvline(limit_qr, color='k', ls='--')
print "The PRTF drops below 1/e at a resolution of %.2f nm" %(1./limit_qr)

In [ ]:
a = prtf_radial[:-1]
print a.reshape((32,2)).mean(axis=1)

In [ ]:
# Loading the signal (assembled)
filename_signal = "../analysis/signal/signal_assembled.h5"
with h5py.File(filename_signal, 'r') as f:
    signal_data = np.rot90(f['data/data'][:], k=2)
    signal_mask = ~np.rot90(f['data/mask'][:].astype(np.bool), k=2)  
    signal_diameter = f['diameter'][...]
    signal_intensity = f['intensity'][...]
    signal_id = f['id'][...]

In [ ]:
# Downsampling
downsampling = 4
intensities, mask = spimage.downsample(signal_data, downsampling, mask=signal_mask, mode='integrate')

In [ ]:
# Cropping
cropping = 154
cx, cy = -2,-6
intensities_cropped = intensities[cropping+cy:-cropping+cy-1,cropping+cx:-cropping+cx-1]
mask_cropped = mask[cropping+cy:-cropping+cy-1,cropping+cx:-cropping+cx-1]
sh = intensities_cropped.shape
print "Shape after downsampling and cropping: ", sh

In [ ]:
mask_prtf = mask_cropped & np.rot90(np.rot90(mask_cropped))

In [ ]:
mask_test = ndimage.gaussian_filter(prtf_2d,2)>1./np.e

In [ ]:
plot = plotting.Plot(fontsize=8, axes_visible=False, save_png=True)
plot.title_label = ['']
plot.xlabel = ['']
plot.ylabel = ['']
plot.plotting_a_map(0, np.abs(fourier[-1])**2, 
                    log=True, mask=mask_test, bad='0.8', under='w', vmin=1e-5, vmax=100, cmap='BuPu')
plot.show()

In [ ]:
plotting.plt.figure(figsize=(20,20))
plotting.plt.scatter(rerror[valid], ferror[valid])
plotting.plt.axhline(0.2151)
plotting.plt.axvline(0.1402)
#plotting.plt.xlim([])
plotting.plt.show()

In [ ]:

plotting.plt.figure(figsize=(20,20))
plotting.plt.hist2d(rerror, ferror, bins=100, norm=plotting.colors.LogNorm(), cmap='hot')
plotting.plt.colorbar()
plotting.plt.show()

In [ ]:
prtf_radial

In [ ]:
def rebin( a, newshape ):
        '''Rebin an array to a new shape.
        '''
        assert len(a.shape) == len(newshape)

        slices = [ slice(0,old, float(old)/new) for old,new in zip(a.shape,newshape) ]
        coordinates = np.mgrid[slices]
        indices = coordinates.astype('i')   #choose the biggest smaller integer index
        return a[tuple(indices)]

In [ ]:
print type(prtf_radial[:-1])
print rebin(prtf_radial[:-1], (32,))